### Establish Snowpark connection and load tables from source data.

This notebook is based on example described in [Building and deploying a time series forecast with Hex + Snowflake](https://quickstarts.snowflake.com/guide/hex/index.html#0). This entire example higlights how we can use Snowflake to perform parallel hyperparameter tuning forecasting foot traffic. Please take a look at Chase Romano's article [Parallel Hyperparameter tuning using Snowpark](https://medium.com/snowflake/parallel-hyperparameter-tuning-using-snowpark-53cdec2faf77) for more information.

We will begin by establishing our Snowflake connection and Snowpark session. This demo assumes the user has access to the `SYSADMIN` role and a virtual warehouse named `COMPUTE_WH` exists and is available for usage. 

In the event the database or schema does not exist, the connection will be established without database and schema context, but we will create them in this notebook. 

In [1]:
from snowflake.snowpark import Session
from snowflake.snowpark.types import StringType
from snowflake.snowpark.functions import to_date, cast, to_timestamp
import pandas as pd
import os

connection_params = {
    "account": os.environ.get("SNOWFLAKE_ACCOUNT"),
    "user": os.environ.get("SNOWFLAKE_USER"),
    "password": os.environ.get("SNOWFLAKE_PASSWORD"),
    "database": os.environ.get("SNOWFLAKE_DATABASE"),
    "schema": os.environ.get("SNOWFLAKE_SCHEMA"),
    "role": "SYSADMIN",
    "warehouse": "COMPUTE_WH",
}

session = Session.builder.configs(connection_params).create()

We connected earlier using the SYSADMIN role and a virtual warehouse named COMPUTE_WH. Let's create a new database and schema in the event that they do not already exist.

In [2]:
session.sql(
    f"CREATE DATABASE IF NOT EXISTS {os.environ.get('SNOWFLAKE_DATABASE')}"
).collect()
session.sql(
    f"CREATE SCHEMA IF NOT EXISTS {os.environ.get('SNOWFLAKE_DATABASE')}.{os.environ.get('SNOWFLAKE_SCHEMA')}"
).collect()
session.sql(f"USE DATABASE {os.environ.get('SNOWFLAKE_DATABASE')}").collect()
session.sql(
    f"USE SCHEMA {os.environ.get('SNOWFLAKE_DATABASE')}.{os.environ.get('SNOWFLAKE_SCHEMA')}"
).collect()

[Row(status='Statement executed successfully.')]

I'm going to create two Pandas DataFrames based on some CSV files that I have available. These files were generated using a process described in [Building and deploying a time series forecast with Hex + Snowflake](https://quickstarts.snowflake.com/guide/hex/index.html#0). The data is in the `data` directory of this repository.

In [3]:
calendar_df = pd.read_csv("../data/calendar.csv.gz")
traffic_df = pd.read_csv("../data/hourly_traffic.csv.gz")

Let's look at our first Pandas DataFrame.

In [4]:
calendar_df.head(5)

,CALENDAR_DATE,CALENDAR_WEEK_DAY_NBR,CALENDAR_MTH_DAY_NBR,CALENDAR_MTH,CALENDAR_YEAR,HOLIDAY_NAME
0,2018-01-01,0,1,1,2018,New Year's Day
1,2018-01-02,1,2,1,2018,NaN
2,2018-01-03,2,3,1,2018,NaN
3,2018-01-04,3,4,1,2018,NaN
4,2018-01-05,4,5,1,2018,NaN


Let's get some information and describe both of these tables to see what we're working with.

In [5]:
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2922 entries, 0 to 2921
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   CALENDAR_DATE          2922 non-null   object
 1   CALENDAR_WEEK_DAY_NBR  2922 non-null   int64 
 2   CALENDAR_MTH_DAY_NBR   2922 non-null   int64 
 3   CALENDAR_MTH           2922 non-null   int64 
 4   CALENDAR_YEAR          2922 non-null   int64 
 5   HOLIDAY_NAME           85 non-null     object
dtypes: int64(4), object(2)
memory usage: 137.1+ KB


We can adjust those "object" types to be more specific.

In [6]:
calendar_df["CALENDAR_DATE"] = pd.to_datetime(calendar_df["CALENDAR_DATE"])
calendar_df["HOLIDAY_NAME"] = calendar_df["HOLIDAY_NAME"].astype("string")

As of the time of this writing, the Snowpark DataFrame from Pandas method converts `datetime64[ns]` to `LongType()` Snowpark types representing [unix time](https://en.wikipedia.org/wiki/Unix_time). We can convert this specific column to make it easier to work with inside of Snowflake. We understand this to be a generic date, so that is what we will convert it to with the `to_date` function. 

Let's persist this table in Snowflake.

I'm using the `overwrite` mode here, but in a typical workflow you would likely want to append to the table.

In [7]:
session.create_dataframe(calendar_df).with_column(
    "CALENDAR_DATE", to_date(cast("CALENDAR_DATE", StringType()))
).write.save_as_table("CALENDAR_INFO", mode="overwrite")

create_temp_table is deprecated. We still respect this parameter when it is True but please consider using `table_type="temporary"` instead.


Let's peek at our table. We can also view the schema to see that the `CALENDAR_DATE` column is now a `DATE` type.

In [8]:
session.table("CALENDAR_INFO").show()

--------------------------------------------------------------------------------------------------------------------------
|"CALENDAR_WEEK_DAY_NBR"  |"CALENDAR_MTH_DAY_NBR"  |"CALENDAR_MTH"  |"CALENDAR_YEAR"  |"HOLIDAY_NAME"  |"CALENDAR_DATE"  |
--------------------------------------------------------------------------------------------------------------------------
|0                        |1                       |1               |2018             |New Year's Day  |2018-01-01       |
|1                        |2                       |1               |2018             |NULL            |2018-01-02       |
|2                        |3                       |1               |2018             |NULL            |2018-01-03       |
|3                        |4                       |1               |2018             |NULL            |2018-01-04       |
|4                        |5                       |1               |2018             |NULL            |2018-01-05       |
|5              

Now for our other table for hourly traffic.

In [9]:
traffic_df.head()

,TIME_POINTS,HOURLY_TRAFFIC,HOLIDAY_NAME,STORE_ID,COLLEGE_TOWN
0,2018-06-16 07:00:00,65,NaN,1,0
1,2018-06-16 08:00:00,58,NaN,1,0
2,2018-06-16 09:00:00,62,NaN,1,0
3,2018-06-16 10:00:00,67,NaN,1,0
4,2018-06-16 11:00:00,74,NaN,1,0


In [10]:
traffic_df.describe()

,HOURLY_TRAFFIC,STORE_ID,COLLEGE_TOWN
count,5.284600e+06,5.284600e+06,5.284600e+06
mean,4.222849e+01,1.005000e+02,5.150000e-01
std,2.990801e+01,5.773431e+01,4.997750e-01
min,0.000000e+00,1.000000e+00,0.000000e+00
25%,1.700000e+01,5.075000e+01,0.000000e+00
50%,3.100000e+01,1.005000e+02,1.000000e+00
75%,6.600000e+01,1.502500e+02,1.000000e+00
max,1.350000e+02,2.000000e+02,1.000000e+00


In [11]:
traffic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5284600 entries, 0 to 5284599
Data columns (total 5 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   TIME_POINTS     object
 1   HOURLY_TRAFFIC  int64 
 2   HOLIDAY_NAME    object
 3   STORE_ID        int64 
 4   COLLEGE_TOWN    int64 
dtypes: int64(3), object(2)
memory usage: 201.6+ MB


STORE_ID and COLLEGE_TOWN probably need some adjustments, I don't imagine these columns will need to store numbers up to 9,223,372,036,854,775,807. Let's make them `int16` and `bool` respectively.

We will similar conversion as we did with the previous DataFrame. For our time conversion, the `to_datetime` function will still let us use the hour value in the `TIME_POINTS` column.

In [12]:
traffic_df["STORE_ID"] = pd.to_numeric(traffic_df["STORE_ID"], downcast="signed")
traffic_df["COLLEGE_TOWN"] = traffic_df["COLLEGE_TOWN"].astype("boolean")
traffic_df["TIME_POINTS"] = pd.to_datetime(traffic_df["TIME_POINTS"])
traffic_df["HOLIDAY_NAME"] = traffic_df["HOLIDAY_NAME"].astype("string")

In [13]:
traffic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5284600 entries, 0 to 5284599
Data columns (total 5 columns):
 #   Column          Dtype         
---  ------          -----         
 0   TIME_POINTS     datetime64[ns]
 1   HOURLY_TRAFFIC  int64         
 2   HOLIDAY_NAME    string        
 3   STORE_ID        int16         
 4   COLLEGE_TOWN    boolean       
dtypes: boolean(1), datetime64[ns](1), int16(1), int64(1), string(1)
memory usage: 141.1 MB


Yay, less memory. 🎉 Our memory usage in this example went from 201.6+ MB to 141.1 MB. 

Finally, we'll create our Snowflake table.

In [14]:
session.create_dataframe(traffic_df).with_column(
    "TIME_POINTS", to_timestamp(cast("TIME_POINTS", StringType()))
).write.save_as_table("HOURLY_TRAFFIC", mode="overwrite")

Let's preview our table.

In [15]:
session.table("HOURLY_TRAFFIC").show(20)

-----------------------------------------------------------------------------------------
|"HOURLY_TRAFFIC"  |"HOLIDAY_NAME"  |"STORE_ID"  |"COLLEGE_TOWN"  |"TIME_POINTS"        |
-----------------------------------------------------------------------------------------
|65                |NULL            |1           |False           |2018-06-16 07:00:00  |
|58                |NULL            |1           |False           |2018-06-16 08:00:00  |
|62                |NULL            |1           |False           |2018-06-16 09:00:00  |
|67                |NULL            |1           |False           |2018-06-16 10:00:00  |
|74                |NULL            |1           |False           |2018-06-16 11:00:00  |
|75                |NULL            |1           |False           |2018-06-16 12:00:00  |
|81                |NULL            |1           |False           |2018-06-16 13:00:00  |
|29                |NULL            |1           |False           |2018-06-16 14:00:00  |
|31       